In [7]:
import os
import re
import pandas as pd
import spacy
from spacy import displacy
try:
    from db import db
    nlp = spacy.load(r"./model/nl_vacature_sm") 
except ModuleNotFoundError:
    #print(os.path.abspath(os.curdir))
    os.chdir('..') # set werkdirectory 1 niveau terug naar root folder
    from db import db
    nlp = spacy.load(r"./model/nl_vacature_sm") 

In [8]:
# 1697 476 780
sql = """
SELECT 
txt.doc_id, c.job_title, txt.text
FROM     
doc_texts txt
INNER JOIN dbo.doc_language l ON txt.doc_id = l.doc_id and l.language_code = 'nl'
inner join crawled_details c on txt.doc_id = c.doc_id
WHERE 
txt.location = 'body';
"""
texten = db.read_sql(sql)

In [9]:
# https://digitalsynopsis.com/design/beautiful-color-gradients-backgrounds/
colors = {"VAKTERM": "linear-gradient(90deg, #a1c4fd, #c2e9fb)",
         "SOFTWARE": "linear-gradient(90deg, #06beb6, #48b1bf)",
         "GPE": "linear-gradient(90deg, #ffc3a0, #ffafbd)",
         "PERSON": "linear-gradient(90deg, #f5f7fa, #c3cfe2)",
         "MONEY": "linear-gradient(90deg, #f5f7fa, #c3cfe2)"}
options = {"ents": ["VAKTERM","SOFTWARE","GPE","PERSON","MONEY"], "colors": colors}

In [11]:
for row in texten.sample(1).itertuples():
    text = re.sub(r'\s+', ' ', row.text) # vervang wit door een enkele spatie
    doc = nlp(text)
    doc.user_data["title"] = str(row.doc_id) +": " + row.job_title
    displacy.render(doc, style="ent",jupyter=True, options=options)